<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Object orientation part 2: Inheritance</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

# Object Orientation Part 2

 - In the previous lesson we got to know classes, one of the basic building blocks of object-oriented programming
 - In this chapter we will consider inheritance.

## Inheritance

In [ ]:
import random
from typing import Tuple


class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point({self.x:.1f}, {self.y:.1f})"

    def move(self, dx=0, dy=0):
        self.x += dx
        self.y += dy

    def randomize(self):
        self.x = random.gauss(2, 4)
        self.y = random.gauss(3, 2)

In [ ]:
p = Point(0, 0)
p

In [ ]:
assert p.x == 0.0
assert p.y == 0.0

In [ ]:
p.move(2, 3)
p

In [ ]:
assert p.x == 2.0
assert p.y == 3.0

In [ ]:
p.randomize()
p

How can we introduce colored points without having to re-implement the entire functionality of `Point`?

In [ ]:
class ColorPoint(Point):
    def __init__(self, x=0, y=0, color="black"):
        super().__init__(x, y)
        self.color = color

    def __repr__(self):
        return f"ColorPoint({self.x:.1f}, {self.y:.1f}, {self.color!r})"

    def randomize(self):
        super().randomize()
        self.color = random.choice(["black", "red", "green", "blue", "yellow", "white"])

In [ ]:
cp = ColorPoint(2, 3, "red")
cp

In [ ]:
assert cp.x == 2.0
assert cp.y == 3.0
assert cp.color == "red"

In [ ]:
cp.move(2, 3)
cp

In [ ]:
assert cp.x == 4.0
assert cp.y == 6.0
assert cp.color == "red"

In [ ]:
cp.randomize()
cp

## Mini workshop

 - Notebook `workshop_190_inheritance`
 - Section "Inheritance"

## Abstract classes

- Classes that cannot have direct instances
- Have `abc.ABC` as base class
    - (a metaclass is actually responsible for their behavior)
- Allow use of the `@abstractmethod` decorator to define abstract methods
    - Often the body of an abstract method is written as `...`
- Abstract classes that have only abstract methods are called Interfaces
    - Interfaces describe requirements placed on subclasses

In [ ]:
...

In [ ]:
from abc import ABC, abstractmethod

class MyBase(ABC):
    @abstractmethod
    def my_method(self):
        ...

In [ ]:
class MyClass(MyBase):
    def my_method(self):
        super().my_method()
        print("my_method()")

In [ ]:
mc = MyClass()
mc.my_method()

- Abstract methods can provide an implementation
- Classes that inherit from an abstract class but do not override all abstract methods are themselves abstract.

In [ ]:
from abc import ABC, abstractmethod

class MyBase(ABC):
    @abstractmethod
    def my_method(self):
        print("Hi!")

In [ ]:
class MyClass(MyBase):
    pass

In [ ]:
# mc = MyClass()

In [ ]:
class YourClass(MyBase):
    def my_method(self):
        super().my_method()
        print("Hello!")

In [ ]:
yc = YourClass()
yc.my_method()

# Workshop

See `workshop_950_rpg_dice` to `Factory for RPG Cubes`.

## RPG dice

In roleplaying games, conflicts between players are often decided by rolling
dice, often multiple dice at the same time. Furthermore games often use
not only the well known 6-sided dice, but also 4-sided, 8-sided, 20-sided dice, etc.

The number and type of dice is described by the following notation:

```text
<number of dice> d <number of sides per die>
```

For example, rolling two 6-sided dice is described as `2d6`.
Sometimes more complex formulas are used: `3d20 + 2d6 - 4`
means that three 20-sided dice and two 6-sided dice are rolled
at the same time, and the total sum of numbers is then reduced by 4.

In some games, rolling the lowest or highest number of dice is treated
in a special way ("catastrophic failure", "critical success").

In the following exercise your task is to implement RPG dice in Python.
To simplify testing your implementation it might be advisable
to implement it in an IDE, but it is also possible to write tests as
assertions in a jupyter notebook. 

Write tests for each functionality you implement. How can you deal with the
randomness in dice rolling? What are the strengths and weaknesses of the strategy
you have chosen to test?

# Protocols

With protocols Python supports structural subtyping, i.e., the derivation of subtype relationships from the structure of classes (in contrast to nominal subtyping where the relationships have to be decalared via inheritance).

In [ ]:
from typing import Protocol, runtime_checkable, SupportsInt

In [ ]:
class MyNumber:
    def __int__(self):
        return 0

In [ ]:
my_number = MyNumber()
int(my_number)

In [ ]:
isinstance(MyNumber, SupportsInt)

In [ ]:
@runtime_checkable
class SupportsCastSpell(Protocol):
    def cast_spell(self, name):
        ...

In [ ]:
@runtime_checkable
class SupportsHit(Protocol):
    def hit(self, who, how):
        ...

In [ ]:
class Mage:
    def __init__(self, name="The Mage"):
        self.name = name
    def cast_spell(self, spell):
        print(f"{self.name} casts a {spell} spell.")

In [ ]:
class Fighter:
    @property
    def name(self):
        return "The Fighter"
    def hit(self, opponent, weapon):
        print(f"{self.name} attacks {opponent} with {weapon}.")

In [ ]:
class Bard:
    def __init__(self, name="The Bard"):
        self.name = name

In [ ]:
p1 = Mage()
p2 = Fighter()
p3 = Bard()

In [ ]:
isinstance(p1, SupportsCastSpell)

In [ ]:
isinstance(p2, SupportsCastSpell)

In [ ]:
isinstance(p3, SupportsCastSpell)

In [ ]:
isinstance(p1, SupportsHit)

In [ ]:
isinstance(p2, SupportsHit)

In [ ]:
isinstance(p3, SupportsHit)

In [ ]:
@runtime_checkable
class HasName(Protocol):
    @property
    def name(self):
        ...

In [ ]:
isinstance(p1, HasName)

In [ ]:
isinstance(p2, HasName)

In [ ]:
isinstance(p3, HasName)

## Mini workshop

 - Notebook `workshop_190_inheritance`
 - Section "Protocols"

## Single dispatch functions

Single dispatch functions allow "methods" to be defined outside of classes, i.e. one can define functions that are polymorphic in their first argument.

This mechanism allows the flexible extension of already existing classes.

In [ ]:
from functools import singledispatch

In [ ]:
@singledispatch
def attack(player: HasName, opponent):
    print(f"{player.name} just stares at the carnage.")

In [ ]:
@attack.register
def _(player: Mage, opponent):
    player.cast_spell("fireball")

In [ ]:
@attack.register
def _(player: Fighter, opponent):
    player.hit(opponent, "sword")

In [ ]:
attack(p1, "The Baddie")

In [ ]:
attack(p2, "The Baddie")

In [ ]:
attack(p3, "The Baddie")

## Multiple inheritance

In [ ]:
class A:
    """Superclass of everything"""
    def __init__(self, arg_a="arg_a", **kwargs):
        super().__init__(**kwargs)
        print(f"__init__(A, {arg_a})")
    
    def f(self):
        print(f"f(A) on {self!r}")

    def g(self):
        print(f"g(A) on {self!r}")

In [ ]:
class B(A):
    def __init__(self, arg_b="arg_b", **kwargs):
        super().__init__(**kwargs)
        print(f"__init__(B, {arg_b})")

    def f(self):
        print(f"f(B) on {self!r}")
        super().f()

    def g(self):
        print(f"g(B) on {self!r}")
        A.g(self)

In [ ]:
class C(A):
    def __init__(self, arg_c="arg_c", **kwargs):
        super().__init__(**kwargs)
        print(f"__init__(C, {arg_c})")
    
    def f(self):
        print(f"f(C) on {self!r}")
        super().f()

    def g(self):
        print(f"g(C) on {self!r}")
        A.g(self)

In [ ]:
class D(B, C):
    def __init__(self, arg_d="arg_d", **kwargs):
        super().__init__(**kwargs)
        print(f"__init__(D, {arg_d})")
    
    def f(self):
        print(f"f(D) on {self!r}")
        super().f()

    def g(self):
        print(f"g(D) on {self!r}")
        B.g(self)
        C.g(self)

In [ ]:
d = D()
d.f()

In [ ]:
d.g()

In [ ]:
type(d).mro()